# streamを理解したい

In [1]:
from langchain_core.runnables import (
    Runnable,
    RunnableGenerator,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.messages.ai import AIMessageChunk
from typing import Iterator
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
chain = RunnablePassthrough()

for chunk in chain.stream("hello"):
    print(chunk, end="|")

print()
print(chain.invoke("hello"), end="|")

hello|
hello|

In [4]:
def gen(iter: Iterator[str]) -> Iterator[str]:
    for s in iter:
        for c in s:
            yield c


chain = RunnableGenerator(gen)

for chunk in chain.stream("hello"):
    print(chunk, end="|")

print()
print(chain.invoke("hello"), end="|")

h|e|l|l|o|
hello|

In [5]:
model = ChatOpenAI()

for chunk in model.stream("日本の首都は？"):
    print(chunk, end="|")

print()

answer = model.invoke("日本の首都は？")
print(answer, end="|")

content=''|content='東'|content='京'|content='です'|content='。'|content=''|
content='東京です。'|

In [6]:
model = ChatOpenAI()


def gen(chunk: AIMessageChunk) -> Iterator[any]:
    yield chunk.content


chain = model | RunnableLambda(gen)

for chunk in chain.stream("日本の首都は？"):
    print(chunk, end="|")

print()

answer = chain.invoke("日本の首都は？")
print(answer, end="|")

東京です。|
東京です。|

In [7]:
model = ChatOpenAI()


def gen(iter: Iterator[AIMessageChunk]) -> Iterator[any]:
    for c in iter:
        yield c.content


chain = model | RunnableGenerator(gen)

for chunk in chain.stream("日本の首都は？"):
    print(chunk, end="|")

print()

answer = chain.invoke("日本の首都は？")
print(answer, end="|")

|東|京|です|。||
東京です。|

In [8]:
def gen1(iter: Iterator[str]) -> Iterator[str]:
    for s in iter:
        for c in s:
            print(f"<{c}>", end="")
            yield c


def gen2(iter: Iterator[any]):
    for s in iter:
        print(f"[{s}]", end="")
        yield s


chain = RunnableGenerator(gen1) | RunnableGenerator(gen2)

for chunk in chain.stream("hello"):
    print(f"({chunk})", end="")

print()

answer = chain.invoke("hello")
print(f"({answer})")

<h>[h](h)<e>[e](e)<l>[l](l)<l>[l](l)<o>[o](o)
<h><e><l><l><o>[hello](hello)
